# What is a Scenario?

A scenario is a set of parameters that define a market condition. Those parameters drive the factor returns that define
that scenario, and we can then use those factor returns, along with the current factor exposures of a portfolio,
to calculate the theoretical factor performance.

What Types of Scenarios Exist?

## Custom Factor Shocks

This type of scenario shocks explicitly defined risk model factors up or down a certain percent. These scenarios must be risk-model specific and have two variations:

1. **No Propagation**: This means the factors are shocked exactly as specified
2. **Propagation**: all the factor shocks defined in the scenario are applied normally, but we also shock all other factors by a calculable amount as defined by the correlation between that factor and the factors that are shocked

### How do I create a custom factor shock scenario?

Let's create a custom factor shock scenario in which the Value factor in the global Axioma risk model rises by 5%,
the Growth factor moves down by -5%, and the Medium-Term Momentum drops by -2%.

#### I. Initialize a GsSession

The initial step is to import all required modules and initialize a GsSession with your application ID and secret.

In [1]:
from gs_quant.session import GsSession, Environment
from gs_quant.markets.scenario import FactorScenario, FactorShockParameters, FactorShock, FactorScenarioType, \
    HistoricalSimulationParameters
from gs_quant.entities.entity import ScenarioCalculationMeasure
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.entities.entitlements import Entitlements, EntitlementBlock, User
import datetime as dt

GsSession.use(Environment.PROD, client_id=None, client_secret=None)

#### II. Define custom factor shocks parameters

Next, we define the set of parameters that encapsulate the key components of our custom factor shock scenario:
 - A valid risk model ID (see [full list of available risk models here](https://marquee.gs.com/s/discover/data-services/catalog?Category=Factor+Risk+Model)).
 - A set of factors and their corresponding shocks in %, defined as a list of `FactorShock` object.
 - Whether these shocks are propagated according to the correlation of factors in the risk model

These three components will be encapsulated in a `FactorShockParameters` object. You make take a quick look at [historical factor returns](https://developer.gs.com/p/docs/services/risk/factor-models/get-factor-model-data/#:~:text=Get%20All%20Factor%20Returns) to guide you in
determining factor shocks. Below, we are defining two sets of factor shock parameters with similar attributes
except for the `propagate_shocks` component.

In [2]:
risk_model_id = "AXIOMA_AXWW4M"
factor_1 = "Value"
factor_2 = "Growth"
factor_3 = "Medium-Term Momentum"

factor_shocks = [
    FactorShock(factor=factor_1, shock=5),
    FactorShock(factor=factor_2, shock=-5),
    FactorShock(factor=factor_3, shock=-2)
]


factor_shock_parameters_no_propagation = FactorShockParameters(factor_shocks=factor_shocks,
                                                               risk_model=risk_model_id,
                                                               propagate_shocks=False)

factor_shock_parameters_with_propagation = FactorShockParameters(factor_shocks=factor_shocks,
                                                                 risk_model=risk_model_id,
                                                                 propagate_shocks=True)

#### III. Set Scenario entitlements

By default, an application will have all entitlement permissions to a scenario it creates. If you would like
to share the scenario with other Marquee users at your firm or other applications, you will need to specify them in the
 `entitlements` attribute of the scenario. Let's walk through how we convert a list of admin, edit and viewer emails
 into an `Entitlements` object:

In [14]:
admin_emails = ["ADMIN EMAILS"]
edit_emails = ["EDIT EMAILS"]
view_emails = ["VIEW EMAILS"]

admin_entitlements = EntitlementBlock(users=User.get_many(emails=admin_emails))
edit_entitlements = EntitlementBlock(users=User.get_many(emails=edit_emails))
view_entitlements = EntitlementBlock(users=User.get_many(emails=view_emails))

scenario_entitlements = Entitlements(
    view=view_entitlements,
    edit=edit_entitlements,
    admin=admin_entitlements
)

#### IV. Create a custom factor shock scenario

Now we are ready to create a factor shock scenario. We will need:

- A name for the scenario
- The type of the scenario. Since we are creating a factor shock scenario, the type will be `FactorScenarioType.Factor_Shock`
- The parameters of the scenario
- Scenario entitlements
- A description for the scenario

Below, we will create two scenarios: one factor shock scenario with no propagation, the second one with propagation.

In [13]:
factor_shock_scenario_no_propagation_name = f"Sample factor shock scenario with {risk_model_id} with no propagation"
factor_shock_scenario_no_propagation = FactorScenario(name=factor_shock_scenario_no_propagation_name,
                                                      type=FactorScenarioType.Factor_Shock,
                                                      parameters=factor_shock_parameters_no_propagation,
                                                      description=f"Sample custom shock scenario with {risk_model_id}",
                                                      entitlements=scenario_entitlements)
factor_shock_scenario_no_propagation.save()
print(factor_shock_scenario_no_propagation)

factor_shock_scenario_propagation_name = f"Sample factor shock scenario with {risk_model_id} with propagation"
factor_shock_scenario_with_propagation = FactorScenario(name=factor_shock_scenario_propagation_name,
                                                        type=FactorScenarioType.Factor_Shock,
                                                        parameters=factor_shock_parameters_with_propagation,
                                                        description=f"Sample custom shock scenario with {risk_model_id}",
                                                        entitlements=scenario_entitlements)
factor_shock_scenario_with_propagation.save()
print(factor_shock_scenario_with_propagation)

## Historical Factor Scenario

In a historical factor scenario, the aim is to replicate the market conditions during a significant or
topical historical event and estimate the impact on your portfolio today. The first step is to define the time period
 of the historical event. Then, factor returns across the time period are geometrically aggregated, and these aggregated
 returns are the factor shocks that will be used to derive the estimated impact on the performance of your portfolio.


### How do I create a historical factor scenario?

Similar to factor shock scenarios, we will need to provide:
- A name for the scenario
- The type of the scenario. In this case, the scenario type is `FactorScenarioType.Factor_Historical_Simulation`
- The parameters of the scenario
- Scenario entitlements
- A description for the scenario

The parameters are defined in a `HistoricalSimulationParameters` object and will define the time
period of the historical simulation scenario:

- Start Date: The start date of the historical event simulation period
- End Date: The end date of the historical event simulation period.


In [6]:
start_date = dt.date(2023, 1, 1)
end_date = dt.date(2024, 1, 1)

date_range_parameters = HistoricalSimulationParameters(start_date=start_date, end_date=end_date)
historical_scenario_name = f"Example historical simulation scenario from " \
                           f"{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}"
historical_replay_scenario = FactorScenario(name=historical_scenario_name,
                                            type=FactorScenarioType.Factor_Historical_Simulation,
                                            parameters=date_range_parameters,
                                            description=f"Sample historical simulation scenario from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}",
                                            entitlements=scenario_entitlements)

historical_replay_scenario.save()
print(historical_replay_scenario)

## How do I get my scenarios or scenarios that are shared with me?

You can get saved scenarios that you created or that were shared with you:
- By its unique Marquee ID
- By its name
- By applying filters such as `risk_model`, `type` of the scenario, the `shocked_factors`, or whether the shocks are propagated
- For a historical simulation scenario, you can filter by a `start_date` and `end_date`.

In [8]:
# Get by id
scenario_by_id = FactorScenario.get(historical_replay_scenario.id)

# Get all my factor Shock scenarios (and those that are shared with me) with risk model AXIOMA_AXWW4M and whose
# factor shocks are propagated to all factors
many_scenarios = FactorScenario.get_many(risk_model=risk_model_id,
                                         propagated_shocks=True,
                                         type=FactorScenarioType.Factor_Shock)
# Get all my factor Shock scenarios (and those that are shared with me) with risk model AXIOMA_AXWW4M that shocks the
# Value factor
many_scenarios = FactorScenario.get_many(risk_model=risk_model_id,
                                         shocked_factors=["Value"],
                                         type=FactorScenarioType.Factor_Shock)

# Get all my historical simulation scenarios that are within the requested start and end date
historical_simulation_scenarios = FactorScenario.get_many(type="Factor Historical Simulation",
                                                          start_date=dt.date(2023, 1, 1),
                                                          end_date=dt.date(2024, 1, 1))

## How do I get GS pre-canned scenarios?

To get GS pre-canned scenarios, you can additionally filter by the tag "GS". See below for some examples

In [15]:
# Get 3 factor shocks scenarios with risk model AXIOMA_AXWW4M that propagate factor shocks
pre_canned_factor_shock_scenarios = FactorScenario.get_many(risk_model=risk_model_id,
                                                            propagated_shocks=True,
                                                            type=FactorScenarioType.Factor_Shock,
                                                            tags=["GS"],
                                                            limit=3)
print(pre_canned_factor_shock_scenarios)

# Get 3 historical simulation that are within selected start and end date
pre_canned_historical_simulation_scenarios = FactorScenario.get_many(type="Factor Historical Simulation",
                                                                     start_date=dt.date(2010, 1, 1),
                                                                     end_date=dt.date(2024, 1, 1),
                                                                     tags=["GS"],
                                                                     limit=3)
print(pre_canned_historical_simulation_scenarios)

## Running scenario(s) on a portfolio

To see the estimated impact of scenarios on your portfolio, run the `get_factor_scenario_analytics` function in the
`PortfolioManager` class, which is the interface for managing your portfolios in Marquee.

A scenario calculation request requires the following components:

- A date: date on which to run a scenario. We will run the scenario(s) on your portfolio holdings as of this date.
- Scenarios: List of scenarios to run. Can either be a list of unique Marquee IDs, or a scenario object itself.
- Risk Model: The risk model to run a scenario with.
- Measures: Metrics to return, which include:
    1. Total portfolio estimated factor PnL
    2. Total portfolio PnL by GICS sector & industry
    3. Total portfolio PnL by region
    4. Total portfolio PnL by the direction (LONG/SHORT) of your portfolio holdings,
    5. Estimated performance of each asset in your portfolio.

In [11]:
portfolio_id = "PORTFOLIO ID"
pm = PortfolioManager(portfolio_id)

scenario_analytics = pm.get_factor_scenario_analytics(scenarios=[historical_replay_scenario],
                                                      date=dt.date(2024, 4, 10),
                                                      measures=[ScenarioCalculationMeasure.SUMMARY,
                                                                ScenarioCalculationMeasure.ESTIMATED_FACTOR_PNL,
                                                                ScenarioCalculationMeasure.ESTIMATED_PNL_BY_SECTOR,
                                                                ScenarioCalculationMeasure.ESTIMATED_PNL_BY_REGION,
                                                                ScenarioCalculationMeasure.ESTIMATED_PNL_BY_DIRECTION,
                                                                ScenarioCalculationMeasure.ESTIMATED_PNL_BY_ASSET],
                                                      risk_model=risk_model_id)
summary = scenario_analytics.get('summary')
factor_pnl = scenario_analytics.get('factorPnl')
sector_aggregations = scenario_analytics.get('bySectorAggregations')
region_aggregations = scenario_analytics.get("byRegionAggregations")
by_asset_aggregations = scenario_analytics.get('byAsset')

## Run scenario(s) on a basket

We can also run one or multiple scenarios on an existing equity custom basket and pull scenario analytics data.

In [ ]:
from gs_quant.markets.baskets import Basket

basket_ticker = "BASKET_TICKER"
basket = Basket.get(basket_ticker)

basket_scenario_analytics = basket.get_factor_scenario_analytics(
    scenarios=[factor_shock_scenario_no_propagation, factor_shock_scenario_with_propagation, historical_replay_scenario],
    date=dt.date(2024, 4, 1),
    measures=[ScenarioCalculationMeasure.SUMMARY, ScenarioCalculationMeasure.ESTIMATED_FACTOR_PNL,
              ScenarioCalculationMeasure.ESTIMATED_PNL_BY_SECTOR,
              ScenarioCalculationMeasure.ESTIMATED_PNL_BY_REGION,
              ScenarioCalculationMeasure.ESTIMATED_PNL_BY_DIRECTION,
              ScenarioCalculationMeasure.ESTIMATED_PNL_BY_ASSET],
   risk_model=risk_model_id)

summary = basket_scenario_analytics.get('summary')
factor_pnl = basket_scenario_analytics.get('factorPnl')
sector_aggregations = basket_scenario_analytics.get('bySectorAggregations')
region_aggregations = basket_scenario_analytics.get("byRegionAggregations")
by_asset_aggregations = basket_scenario_analytics.get('byAsset')


